In [ ]:
import asyncio
import json
import signal
from datetime import datetime
import websockets
from confluent_kafka import Producer
import logging,os
from dotenv import load_dotenv

In [ ]:
load_dotenv()
kafka_bootstrap=os.getenv('KAFKA_BOOTSTRAP_SERVER')
kafka_api_key=os.getenv('KAFKA_API_KEY') # of cluster
kafka_secret_key=os.getenv('KAFKA_SECRET_KEY') # of cluster

In [ ]:
logger=logging.getLogger(__name__)
OKX_WS = "wss://ws.okx.com:8443/ws/v5/public"
TOPIC_TRADES = "okx_trades"
TOPIC_ORDERBOOK = "okx_orderbook"
TOPIC_FUNDING = "okx_funding"
SUBSCRIBE_ARGS = [
    {"channel": "trades", "instId": "BTC-USDT-SWAP"},
    {"channel": "books-l2-tbt", "instId": "BTC-USDT-SWAP"},
    {"channel": "funding-rate", "instId": "BTC-USDT-SWAP"}
]

In [ ]:
configure={
    "bootstrap.servers": kafka_bootstrap,
    'sasl.username': kafka_api_key,
    'sasl.password': kafka_secret_key,
    'security.protocol': 'SASL_SSL',
    'sasl.mechanisms': 'PLAIN', # credential username/password
    'linger.ms': 50,
    'batch.size': 1000
    #"queue.buffering.max.messages": 100000,
    #"buffer.memory": 102400
}
p = Producer(configure)
running = True
# if remain any data shutdown() will push all of this before shutdown
def delivery_callback(err, msg):
    if err is not None:
        logger.error(f'Message delivery failed: {err}')
    else:
        logger.info(f'Message delivered to {msg.topic()} [{msg.partition()}] at offset {msg.offset()}')

def shutdown():
    global running
    running = False
    try:
        p.flush()
    except:
        pass
    print("Producer shutdown complete.")

signal.signal(signal.SIGINT, lambda s, f: shutdown())
signal.signal(signal.SIGTERM, lambda s, f: shutdown()) # stop program and call shutdown() to push remain data

# send ping each 20s ->websocket don't close connect
async def heartbeat(ws):
    while running:
        try:
            await ws.send("ping")
        except:
            break
        await asyncio.sleep(20)


async def route_message(raw_msg):
    """Route OKX WS message to the correct Kafka topic."""
    if raw_msg == "pong":
        return

    try:
        msg = json.loads(raw_msg)
    except:
        msg = {"raw": raw_msg}

    arg = msg.get("arg", {})
    chan = arg.get("channel", "")

    if chan == "trades":
        topic = TOPIC_TRADES
    elif "book" in chan:
        topic = TOPIC_ORDERBOOK
    elif chan == "funding-rate":
        topic = TOPIC_FUNDING
    else:
        topic = TOPIC_TRADES

    record = {
        "received_at": datetime.utcnow().isoformat(),
        "payload": msg
    }
    # khi send msg, nếu ko có poll() bộ nhớ sẽ đầy, khi đầy buffer nó sẽ poll và callback lại để xử lí msg
    try:
        p.produce(topic, json.dumps(record).encode("utf-8"),on_delivery=delivery_callback)
    except BufferError:
        p.poll(0) # return 1 message or callback
        p.produce(topic, json.dumps(record).encode("utf-8"),on_delivery=delivery_callback)

    p.poll(0)   # serve delivery callbacks
async def run_collector():
    global running

    while running:
        try:
            async with websockets.connect(OKX_WS, ping_interval=None, max_size=None) as ws:
                await ws.send(json.dumps({"op": "subscribe", "args": SUBSCRIBE_ARGS}))
                print("Subscribed:", SUBSCRIBE_ARGS)

                hb = asyncio.create_task(heartbeat(ws))

                while running:
                    try:
                        msg = await ws.recv()
                        await route_message(msg)

                    except websockets.ConnectionClosed:
                        print("Websocket closed → reconnecting...")
                        break

                hb.cancel()

        except Exception as e:
            print("WebSocket error → reconnect in 3s:", e)
            await asyncio.sleep(3)
if __name__ == "__main__":
    print("Starting OKX → Kafka (Confluent)...")
    asyncio.run(run_collector())
    shutdown()


In [ ]:
import websocket
import json
import time
import threading
from datetime import datetime, timezone

# --- CẤU HÌNH ---
TARGET_INSTRUMENT = "BTC-USDT-SWAP"
TARGET_TIMEFRAME = "1m"
CHANNEL_NAME = f"candle{TARGET_TIMEFRAME}"

# SỬA LỖI: Dùng URL 'BUSINESS' cho kênh Nến (Candles)
# Public URL: wss://ws.okx.com:8443/ws/v5/public (Tickers, Trades, Books)
# Business URL: wss://ws.okx.com:8443/ws/v5/business (Candles)
SOCKET_URL = "wss://ws.okx.com:8443/ws/v5/business"

def process_candle_data(candle_data):
    """Hàm xử lý dữ liệu nến thô."""
    try:
        ts = candle_data[0]
        open_price = candle_data[1]
        high_price = candle_data[2]
        low_price = candle_data[3]
        close_price = candle_data[4]
        volume = candle_data[5]
        is_confirmed = candle_data[8]

        dt_object = datetime.fromtimestamp(int(ts) / 1000, tz=timezone.utc)
        time_str = dt_object.strftime('%Y-%m-%d %H:%M:%S')

        status = "🔴 ĐÃ ĐÓNG" if is_confirmed == "1" else "🟢 Đang chạy"
        print(f"[{time_str}] {status} | O: {open_price} | C: {close_price} | Vol: {volume}")

        if is_confirmed == "1":
            print(f"   >>> LƯU DATABASE: Nến {time_str} đã hoàn tất.")
            # save_to_db(...)
    except IndexError:
        pass

def on_message(ws, message):
    """Hàm nhận tin nhắn."""
    try:
        # Xử lý tin nhắn pong (phản hồi heartbeat)
        if message == "pong":
            return

        payload = json.loads(message)

        # Xử lý tin nhắn lỗi
        if payload.get('event') == 'error':
            print(f"!!! LỖI TỪ OKX: {payload.get('msg')} (Code: {payload.get('code')})")
            return

        # Xử lý tin nhắn đăng ký thành công
        if payload.get('event') == 'subscribe':
            print(f"--- Đã đăng ký thành công kênh: {payload.get('arg', {}).get('channel')} ---")
            return

        # Xử lý dữ liệu nến
        if 'data' in payload:
            arg = payload.get('arg', {})
            if arg.get('channel') == CHANNEL_NAME and arg.get('instId') == TARGET_INSTRUMENT:
                for candle in payload['data']:
                    process_candle_data(candle)

    except Exception as e:
        print(f"Lỗi xử lý tin nhắn: {e}")

def on_error(ws, error):
    print(f"Lỗi WebSocket: {error}")

def on_close(ws, close_status_code, close_msg):
    print("--- Kết nối WebSocket đã đóng ---")

def send_heartbeat(ws):
    """Gửi ping mỗi 25 giây để giữ kết nối."""
    while True:
        try:
            if ws.sock and ws.sock.connected:
                ws.send("ping")
                # print("Sent ping") # Bỏ comment nếu muốn debug heartbeat
            time.sleep(25)
        except Exception:
            break

def on_open(ws):
    """Gửi yêu cầu đăng ký kênh khi kết nối mở."""
    print(f"--- Đã kết nối tới Business URL. Đang đăng ký {CHANNEL_NAME}... ---")

    subscribe_msg = {
        "op": "subscribe",
        "args": [
            {
                "channel": CHANNEL_NAME,
                "instId": TARGET_INSTRUMENT
            }
        ]
    }
    ws.send(json.dumps(subscribe_msg))

    # Bắt đầu luồng gửi heartbeat
    threading.Thread(target=send_heartbeat, args=(ws,), daemon=True).start()

# --- Phần chạy chính ---
if __name__ == "__main__":
    while True: # Vòng lặp tự động kết nối lại (Auto-reconnect)
        try:
            print(f"Đang kết nối tới: {SOCKET_URL}")
            ws = websocket.WebSocketApp(SOCKET_URL,
                                      on_open=on_open,
                                      on_message=on_message,
                                      on_error=on_error,
                                      on_close=on_close)

            ws.run_forever()
        except KeyboardInterrupt:
            print("Đang dừng chương trình...")
            break
        except Exception as e:
            print(f"Lỗi kết nối: {e}")

        print("Mất kết nối. Thử lại sau 5 giây...")
        time.sleep(5)

Đang kết nối tới: wss://ws.okx.com:8443/ws/v5/business
--- Đã kết nối tới Business URL. Đang đăng ký candle1m... ---
--- Đã đăng ký thành công kênh: candle1m ---
[2025-12-04 07:14:00] 🟢 Đang chạy | O: 92907.8 | C: 93015.1 | Vol: 9965.11
[2025-12-04 07:14:00] 🟢 Đang chạy | O: 92907.8 | C: 93015.2 | Vol: 10000.59
[2025-12-04 07:14:00] 🟢 Đang chạy | O: 92907.8 | C: 93015.1 | Vol: 10058.21
[2025-12-04 07:14:00] 🟢 Đang chạy | O: 92907.8 | C: 93015.1 | Vol: 10060.25
[2025-12-04 07:14:00] 🟢 Đang chạy | O: 92907.8 | C: 93015.2 | Vol: 10063.26
[2025-12-04 07:14:00] 🟢 Đang chạy | O: 92907.8 | C: 93015.2 | Vol: 10068.94
[2025-12-04 07:14:00] 🟢 Đang chạy | O: 92907.8 | C: 93015.1 | Vol: 10071.66
[2025-12-04 07:14:00] 🟢 Đang chạy | O: 92907.8 | C: 93005 | Vol: 10104.76
[2025-12-04 07:14:00] 🔴 ĐÃ ĐÓNG | O: 92907.8 | C: 93005 | Vol: 10104.76
   >>> LƯU DATABASE: Nến 2025-12-04 07:14:00 đã hoàn tất.
[2025-12-04 07:15:00] 🟢 Đang chạy | O: 93005 | C: 93000.4 | Vol: 17.43
[2025-12-04 07:15:00] 🟢 Đang chạ